In [ ]:
from networkx import radius
import pandas as pd
import numpy as np

class Node:
    """x, y, function, input0x input0y, input1x, input1y, input2x, input2y, input3x, input3y"""
    def __init__(self, x, y, function, input0x=None, input0y=None, input1x=None, input1y=None, input2x=None, input2y=None, input3x=None, input3y=None):
        self.x = x
        self.y = y
        self.function = function
        self.input0x = input0x
        self.input0y = input0y
        self.input1x = input1x
        self.input1y = input1y
        self.input2x = input2x
        self.input2y = input2y
        self.input3x = input3x
        self.input3y = input3y

    def lut4_to_logic(self) -> str:
        """
        Convert a 16-bit LUT mask into a lambda expression string
        using AND/OR/NOT logic over inputs in3..in0.
        """
        terms = []

        for i in range(16):
            if (self.function >> i) & 1:  # if this input pattern produces a 1
                # Extract bits for in3..in0
                in0 = (i >> 0) & 1
                in1 = (i >> 1) & 1
                in2 = (i >> 2) & 1
                in3 = (i >> 3) & 1

                # Build minterm
                parts = []
                for name, val in zip(["in3", "in2", "in1", "in0"], [in3, in2, in1, in0]):
                    parts.append(name if val else f"(not {name})")

                terms.append("(" + " and ".join(parts) + ")")

        if not terms:
            body = "0"
        elif len(terms) == 1:
            body = terms[0]
        else:
            body = "(\n    " + " or\n    ".join(terms) + "\n)"

        return f"lambda in3, in2, in1, in0: {body}"

        
    def lut4(self):
        """Return a lambda implementing a 4-input LUT function for a 16-bit truth table."""
        return lambda in3, in2, in1, in0: (self.function >> ((in3 << 3) | (in2 << 2) | (in1 << 1) | in0)) & 1

    def __str__(self):
        op = self.lut4_to_logic()
        return f"Node(({self.x}, {self.y}), function={op}, inputs=({self.input0x},{self.input0y}),({self.input1x},{self.input1y}),({self.input2x},{self.input2y}),({self.input3x},{self.input3y}))"
    
    def twos_complement(val, bits):
        """
        Computes the two's complement of a positive integer 'val' 
        for a given number of 'bits'.
        """
        # Calculate the one's complement by inverting all bits
        ones_comp = ~val & ((1 << bits) - 1) 
        # Add 1 to get the two's complement
        twos_comp = ones_comp + 1
        return twos_comp
    
    def to_verilog(self):
        def input_str(x, y):
            if x is None or y is None:
                return "1'b0"
            elif x == 0:
                return f"in{y}"
            else:
                return f"x{x}_y{y}"
        return f"""
SB_LUT4 #(
    .LUT_INIT(16'b{Node.twos_complement(self.function, 16):016b})
) lut_{self.x}_{self.y} (
    .O(x{self.x}_y{self.y}),
    .I0({input_str(self.input0x, self.input0y)}),
    .I1({input_str(self.input1x, self.input1y)}),
    .I2({input_str(self.input2x, self.input2y)}),
    .I3({input_str(self.input3x, self.input3y)})
);"""

    def __repr__(self):
        return self.__str__()

def decode_nodes(arr):
    nodes = []
    # Loop over every 9 elements (each node has 9 values: 1 function + 8 input coordinates)
    for i in range(0, len(arr), 9):
        if i + 8 < len(arr):  # Ensure we have all 9 values
            function = arr[i]
            
            # Extract input coordinates (4 pairs: x,y for each input)
            input0x, input0y = arr[i+1], arr[i+2]
            input1x, input1y = arr[i+3], arr[i+4]
            input2x, input2y = arr[i+5], arr[i+6]
            input3x, input3y = arr[i+7], arr[i+8]
            
            node_index = i // 9
            total_nodes = len(arr) // 9
            # Calculate x, y coordinates based on grid arrangement
            grid_size = int(np.sqrt(total_nodes))
            x = node_index % grid_size
            y = node_index // grid_size
            
            # Create node object
            node = Node(x, y, function, input0x, input0y, input1x, input1y, 
                        input2x, input2y, input3x, input3y)
            
            nodes.append(node)
    
    return nodes

def generate_random_nodes(num_nodes=100):
    """Generate a random array representing nodes for decode_nodes function."""
        
    arr = []
    grid_size = int(np.sqrt(num_nodes))
    
    def generate_random_coords(x, y, x_radius=1, y_radius=2, none_weight=0.25):
        # Generate random coordinates within a given length
        if np.random.rand() < none_weight:
            return None, None
        x_offset = np.random.randint(-x_radius, 0)  # Change 0 to x_radius to allow cycles
        y_offset = np.random.randint(-y_radius, y_radius + 1)
        return max(x + x_offset, 0), max(min(y + y_offset, grid_size - 1), 0)

    for i in range(num_nodes):
        # Random 16-bit function
        function = np.random.randint(0, 2**16)
        
        # Calculate node position in grid
        x = i % grid_size
        y = i // grid_size
        
        if x == 9:
            input0x, input0y = generate_random_coords(x-1, y)
            input1x, input1y = generate_random_coords(x-1, y)
            # input2x, input2y = generate_random_coords(x-1, y)
            # input3x, input3y = generate_random_coords(x-1, y)
        else:
            # Random input coordinates for other nodes
            input0x, input0y = generate_random_coords(x-1, y)
            input1x, input1y = generate_random_coords(x-1, y)
            input2x, input2y = generate_random_coords(x-1, y)
            input3x, input3y = generate_random_coords(x-1, y)
        
        # Add all 9 values for this node (1 function + 8 input coordinates)
        arr.extend([function, input0x, input0y, input1x, input1y, input2x, input2y, input3x, input3y])
        
    return np.array(arr)

arr = generate_random_nodes(100)
# print(arr)
nodes = decode_nodes(arr)
node = nodes[5]
node

Node((2, 1), function=lambda in3, in2, in1, in0: (
    ((not in3) and (not in2) and in1 and in0) or
    ((not in3) and in2 and (not in1) and (not in0)) or
    ((not in3) and in2 and in1 and in0) or
    (in3 and (not in2) and (not in1) and (not in0)) or
    (in3 and (not in2) and in1 and (not in0)) or
    (in3 and in2 and in1 and in0)
), inputs=(0,0),(None,None),(0,2),(None,None))

In [26]:
node.lut4()(0, 0, 1, 0)

0

In [27]:
print(node.to_verilog())


SB_LUT4 #(
    .LUT_INIT(16'b0111101001101000)
) lut_2_1 (
    .O(x2_y1),
    .I0(in0),
    .I1(1'b0),
    .I2(in2),
    .I3(1'b0)
);


In [28]:
def make_verilog(nodes, module_name="cgp_module"):
    outputs = []
    wires = []
    
    for node in nodes:
        if node.x == int(np.sqrt(len(nodes))) - 1:
            outputs.append(f"x{node.x}_y{node.y}")
        else:
            wires.append(f"x{node.x}_y{node.y}")
            
    inputs = [f"in{i}" for i in range(int(np.sqrt(len(nodes))))]
    
    verilog_lines = []
    verilog_lines.append(f"module {module_name} (")
    
    # Declare inputs and outputs
    verilog_lines.append("    input " + ", ".join(inputs) + ",")
    verilog_lines.append("    output " + ", ".join(outputs) + ");\n")
    verilog_lines.append("    wire " + ", ".join(wires) + ";\n")
    
    # Instantiate each node
    for node in nodes:
        verilog_lines.append(node.to_verilog())
    
    verilog_lines.append("\nendmodule")
    
    return "\n".join(verilog_lines)

verilog_code = make_verilog(nodes)
print(verilog_code)

module cgp_module (
    input in0, in1, in2,
    output x2_y0, x2_y1, x2_y2);

    wire x0_y0, x1_y0, x0_y1, x1_y1, x0_y2, x1_y2, x0_y3;


SB_LUT4 #(
    .LUT_INIT(16'b0010101101110000)
) lut_0_0 (
    .O(x0_y0),
    .I0(1'b0),
    .I1(in0),
    .I2(in1),
    .I3(in1)
);

SB_LUT4 #(
    .LUT_INIT(16'b1000001010101100)
) lut_1_0 (
    .O(x1_y0),
    .I0(in1),
    .I1(in2),
    .I2(in1),
    .I3(in0)
);

SB_LUT4 #(
    .LUT_INIT(16'b1101001111001101)
) lut_2_0 (
    .O(x2_y0),
    .I0(1'b0),
    .I1(in1),
    .I2(1'b0),
    .I3(in0)
);

SB_LUT4 #(
    .LUT_INIT(16'b0011001110000001)
) lut_0_1 (
    .O(x0_y1),
    .I0(in0),
    .I1(1'b0),
    .I2(1'b0),
    .I3(in0)
);

SB_LUT4 #(
    .LUT_INIT(16'b0101101001100011)
) lut_1_1 (
    .O(x1_y1),
    .I0(1'b0),
    .I1(in0),
    .I2(in0),
    .I3(in1)
);

SB_LUT4 #(
    .LUT_INIT(16'b0111101001101000)
) lut_2_1 (
    .O(x2_y1),
    .I0(in0),
    .I1(1'b0),
    .I2(in2),
    .I3(1'b0)
);

SB_LUT4 #(
    .LUT_INIT(16'b1101001101001010)
) lut_0_